<a href="https://colab.research.google.com/github/lt911/Applied_Social_network_practice/blob/master/youtube_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [8]:
%ls /content/drive/My\ Drive/project/recommendation_system/

ml-100k.zip


In [13]:
project_fd = "/content/drive/My Drive/project/recommendation_system/"
os.chdir(project_fd)

In [15]:
%ls

ml-100k.zip


In [16]:
# !unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [17]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random 
from tqdm import tqdm_notebook
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_path = "ml-100k/"

In [18]:
device

device(type='cpu')

In [21]:
id_val = 1
num_users = 943
nums_items= 1682
train_df = pd.read_csv(f"{data_path}/u{id_val}.base", 
                              sep='\t', header=None)
train_df.columns=['user_id', 'item_id', 'rating', 'ts']
train_df['user_id'] = train_df['user_id']-1
train_df['item_id'] = train_df['item_id']-1

valid_df = pd.read_csv(f"{data_path}/u{id_val}.test", 
                       sep='\t', header=None)
valid_df.columns=['user_id', 'item_id', 'rating', 'ts']
valid_df['user_id'] = valid_df['user_id']-1
valid_df['item_id'] = valid_df['item_id']-1

train_df.shape, valid_df.shape

((80000, 4), (20000, 4))

In [22]:
train_users = train_df.user_id.unique()
valid_users = valid_df.user_id.unique()

len(train_users), len(valid_users)

(943, 459)

In [24]:
train_items = train_df.item_id.unique()
valid_items = valid_df.item_id.unique()
len(train_items), len(valid_items)

(1650, 1410)

All users are included in training, and 1650/1682 (98.1%) of items are in training. 

## Data

In [25]:
class CollabDataset(Dataset):
  def __init__(self, df, user_col=0, item_col=1, rating_col=2, )